### Machine Learning for Asset Managers

#Chapter 2: Denoising and Detoning

Esse segundo capítulo apresenta uma metodologia parar tratar o ruído e reter o sinal de matrizes de covariância. Esse procedimento é muito útil para obter melhores carteiras com o processo de Markovitz. Para isso, usamos da conclusão do  teorema de Marchenko-Pastur, que diz que uma matriz (NxT) com entradas iid cujo processo gerador dos dados possui média zero e variância definida terá, quando N, T tenderem ao infinito, autovalores que seguirão um dada distribuiço — Marchenko-Pastur.


Além do livro, utilizei essa fonte: https://medium.com/swlh/an-empirical-view-of-marchenko-pastur-theorem-1f564af5603d

In [9]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KernelDensity
from scipy.optimize import minimize
from scipy.linalg import block_diag
from sklearn.covariance import LedoitWolf

In [8]:
def mpPDF(var, q, pts):
    """
    Creates a Marchenko-Pastur Probability Density Function
    Args:
        var (float): Variance
        q (float): T/N where T is the number of rows and N the number of columns
        pts (int): Number of points used to construct the PDF
    Returns:
        pd.Series: Marchenko-Pastur PDF
    """
    # Marchenko-Pastur pdf
    # q=T/N
    # Adjusting code to work with 1 dimension arrays
    if isinstance(var, np.ndarray):
        if var.shape == (1,):
            var = var[0]
    eMin, eMax = var * (1 - (1. / q) ** .5) ** 2, var * (1 + (1. / q) ** .5) ** 2
    eVal = np.linspace(eMin, eMax, pts)
    pdf = q / (2 * np.pi * var * eVal) * ((eMax - eVal) * (eVal - eMin)) ** .5
    pdf = pd.Series(pdf, index=eVal)
    return pdf


In [11]:
def getPCA(matrix):
    """
    Gets the Eigenvalues and Eigenvector values from a Hermitian Matrix
    Args:
        matrix pd.DataFrame: Correlation matrix
    Returns:
         (tuple): tuple containing:
            np.ndarray: Eigenvalues of correlation matrix
            np.ndarray: Eigenvectors of correlation matrix
    """
    # Get eVal,eVec from a Hermitian matrix
    eVal, eVec = np.linalg.eigh(matrix)
    indices = eVal.argsort()[::-1]  # arguments for sorting eVal desc
    eVal, eVec = eVal[indices], eVec[:, indices]
    eVal = np.diagflat(eVal)
    return eVal, eVec